In [2]:
from pathlib import Path
import sagemaker
import boto3

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")

Path("serving").mkdir(parents=True, exist_ok=True)
BASIC_MODEL_PACKAGE_GROUP = "basic-football"

In [3]:
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=BASIC_MODEL_PACKAGE_GROUP,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1,
)

package = (
    response["ModelPackageSummaryList"][0]
    if response["ModelPackageSummaryList"]
    else None
)

package

In [4]:
from sagemaker.s3 import S3Downloader

if package:
    response = sagemaker_client.describe_model_package(
        ModelPackageName=package["ModelPackageArn"],
    )

    model_data = response["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]
    S3Downloader.download(model_data, Path("./").as_posix())

In [8]:
# !python -m flask --app serving/app.py --debug run --host=0.0.0.0 --port=4242

In [10]:
# !curl --location --request POST 'http://localhost:4242/predict' --header 'Content-Type: text/plain' --data-raw '71,71,71,71,71,71,72,70,70,71,74,79,79,79,79,79,79,80,80,71,83,80,1.54,1.61,1.26,0.76,20,28,1.377149515625,2.5561203576634663,71.18,78.91,11.75,14.58,3.55,1.805,53.639,20.03,71.33,78.83,79.0,71.0,71.18181818181819,78.9090909090909,0,0,4.473,29.233606557377048,31.40637450199203,-2.172767944614982'